In [3]:
import numpy as np
import h5py

y_slice_idx = np.linspace(0, 5299, 10, dtype=int)
y_slice_idx

array([   0,  588, 1177, 1766, 2355, 2943, 3532, 4121, 4710, 5299])

In [5]:
path_in = "/net/pasnas01/pool1/enpro-2021-voxie/reconstructed_volumes/zuendkerze/mono/volume.hdf5"

In [4]:
DATATYPE_USED = "float32"
def trans_hdf5_incremental(path_in: str):
    with h5py.File(path_in, "r") as f_in:
        name_old = path_in.split(".hdf5")[0]
        name_new = name_old+"_test_cut.hdf5"
        with h5py.File(name_new, "w") as f_out:
            new_volume_dataset = None

            # takeover all other existing groups beside "Volume"
            for key in f_in.keys():
                if key != "Volume":
                    # Get parent group name for copy
                    group_path = f_in[key].parent.name
                    # Check existence of group, else create group+parent
                    group_id = f_out.require_group(group_path)
                    f_in.copy(key, group_id, group_path+key)
            
            vol_hdf5 = f_in["Volume"]
            # Volume iteration loop
            grey_value_sum = 0
            for y_slice in y_slice_idx:
                vol_data = vol_hdf5[:, y_slice, :]
                grey_value_sum += vol_data.flatten().sum()

                vol_data_3d = np.expand_dims(vol_data, axis=0)
                if new_volume_dataset is None:
                    new_volume_dataset = f_out.create_dataset("Volume",
                        data=vol_data_3d, dtype=DATATYPE_USED, chunks=True,
                        maxshape=(None, None, None))
                else:
                    new_volume_dataset.resize(new_volume_dataset.shape[0] +
                        vol_data_3d.shape[0], axis=0)
                    new_volume_dataset[y_slice, :, :] = vol_data_3d
            
            new_volume_dataset.attrs['MATLAB_class'] = 'double'
            new_volume_dataset.attrs['Mean_grey_value'] = grey_value_sum/vol_hdf5.size